In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from flash_attn import flash_attn_qkvpacked_func
from PIL import Image
import os
import time
from tqdm import tqdm
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

# ✅ Set PyTorch Memory Optimization
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# ✅ Clear Cache Before Running
torch.cuda.empty_cache()

# ✅ Limit Memory Allocation to 80%
torch.cuda.set_per_process_memory_fraction(0.8)

class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        if len(self.image_files) == 0:
            raise ValueError(f"No image files found in {root_dir}")
            
    def __len__(self):
        return len(self.image_files)
        
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

class FlashAttentionBlock(nn.Module):
    def __init__(self, dim, num_heads=8):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        assert dim % num_heads == 0, "dim must be divisible by num_heads!"
        
        self.scale = self.head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim * 3)
        self.proj = nn.Linear(dim, dim)
        self.norm = nn.LayerNorm(dim)
        
    def forward(self, x):
        B, seq_len, C = x.shape
        
        # Apply layer norm first
        x = self.norm(x)
        
        # Project to qkv
        qkv = self.qkv(x)
        
        # Reshape qkv for flash attention
        # Flash attention expects shape: (batch_size, seqlen, 3, num_heads, head_dim)
        qkv = qkv.reshape(B, seq_len, 3, self.num_heads, self.head_dim)
        
        # Flash attention forward pass
        attn_output = flash_attn_qkvpacked_func(
            qkv,  # Shape: [B, seq_len, 3, num_heads, head_dim]
            dropout_p=0.0,
            causal=False
        )
        
        # Reshape output back to original dimensions
        # Flash attention output shape: [B, seq_len, num_heads, head_dim]
        attn_output = attn_output.reshape(B, seq_len, C)
        
        # Final projection
        return self.proj(attn_output)


class I2SBBackbone(nn.Module):
    def __init__(self, in_channels=3, hidden_dim=256, num_blocks=6):  # Reduced num_blocks to save memory
        super().__init__()
        
        self.init_conv = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim // 2, 3, padding=1),
            nn.GroupNorm(8, hidden_dim // 2),
            nn.ReLU(),
            nn.Conv2d(hidden_dim // 2, hidden_dim, 3, padding=1)
        )
        
        self.attention_blocks = nn.ModuleList([
            FlashAttentionBlock(hidden_dim) for _ in range(num_blocks)
        ])
        
        self.ffn_blocks = nn.ModuleList([
            nn.Sequential(
                nn.LayerNorm(hidden_dim),
                nn.Linear(hidden_dim, hidden_dim * 4),
                nn.GELU(),
                nn.Linear(hidden_dim * 4, hidden_dim)
            ) for _ in range(num_blocks)
        ])
        
        self.output_conv = nn.Conv2d(hidden_dim, in_channels, 1)
        
    def forward(self, x):
        x = self.init_conv(x)
        B, C, H, W = x.shape  # Ensure correct dimensions
    
        print(f"📏 Before flatten: {x.shape}")  # Debug output
    
        x = x.flatten(2).transpose(1, 2)  # Flatten to (B, H*W, C)
    
        print(f"📏 After flatten: {x.shape}")  # Debug output
    
        for attn, ffn in zip(self.attention_blocks, self.ffn_blocks):
            x = x + attn(x)
            x = x + ffn(x)
    
        x = x.transpose(1, 2).reshape(B, C, H, W)  # Ensure reshape matches original
    
        print(f"📏 After reshape: {x.shape}")  # Debug output
    
        return self.output_conv(x)


def train():
    # ✅ Hyperparameters
    BATCH_SIZE = 8  # Reduced from 512 to prevent OOM
    LEARNING_RATE = 1e-4
    EPOCHS = 15
    MODEL_PATH = "model-flash.pth"
    DATASET_PATH = r"C:\Users\pc\Downloads\patches"
    VAL_SPLIT = 0.1
    ACCUMULATION_STEPS = 4  # ✅ Added gradient accumulation to reduce memory usage

    # ✅ GPU setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ✅ Transformations
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    dataset = ImageDataset(DATASET_PATH, transform=transform)
    train_size = int((1 - VAL_SPLIT) * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE,
        shuffle=True,
        pin_memory=False,
        num_workers=0  # ✅ Reduced workers to avoid memory issues
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        pin_memory=False,
        num_workers=0
    )

    model = I2SBBackbone().to(device)
    
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.99))
    criterion = nn.MSELoss()
    scaler = torch.amp.GradScaler()

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        torch.cuda.empty_cache()  # ✅ Clears cache each epoch to prevent OOM

        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}") as pbar:
            for batch_idx, batch in enumerate(pbar):
                batch = batch.to(device, non_blocking=True)

                optimizer.zero_grad()

                # ✅ Fix for deprecated autocast warning
                with torch.amp.autocast("cuda"):
                    output = model(batch)
                    loss = criterion(output, batch)

                # ✅ Gradient accumulation (prevents OOM for large batches)
                loss = loss / ACCUMULATION_STEPS
                scaler.scale(loss).backward()

                if (batch_idx + 1) % ACCUMULATION_STEPS == 0:
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()

                total_loss += loss.item()
                pbar.set_postfix({'loss': total_loss / (pbar.n + 1)})

        # ✅ Save model after each epoch
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss / len(train_loader),
        }, MODEL_PATH)

if __name__ == "__main__":
    train()


Epoch 1/15:   0%|                                                                             | 0/3288 [00:00<?, ?it/s]

📏 Before flatten: torch.Size([8, 256, 128, 128])
📏 After flatten: torch.Size([8, 16384, 256])
📏 After reshape: torch.Size([8, 256, 128, 128])


Epoch 1/15:   0%|                                                        | 1/3288 [00:01<1:14:12,  1.35s/it, loss=0.16]

📏 Before flatten: torch.Size([8, 256, 128, 128])
📏 After flatten: torch.Size([8, 16384, 256])
📏 After reshape: torch.Size([8, 256, 128, 128])


Epoch 1/15:   0%|                                                         | 2/3288 [00:02<58:29,  1.07s/it, loss=0.157]

...

📏 Before flatten: torch.Size([8, 256, 128, 128])
📏 After flatten: torch.Size([8, 16384, 256])
📏 After reshape: torch.Size([8, 256, 128, 128])


Epoch 2/15:   6%|███                                                 | 197/3288 [02:51<44:12,  1.17it/s, loss=0.000453]

📏 Before flatten: torch.Size([8, 256, 128, 128])
📏 After flatten: torch.Size([8, 16384, 256])
📏 After reshape: torch.Size([8, 256, 128, 128])


Epoch 2/15:   6%|███▏                                                | 198/3288 [02:52<44:16,  1.16it/s, loss=0.000458]

📏 Before flatten: torch.Size([8, 256, 128, 128])
📏 After flatten: torch.Size([8, 16384, 256])
📏 After reshape: torch.Size([8, 256, 128, 128])


In [14]:
import torch
torch.cuda.empty_cache()


In [2]:
import torch
torch.cuda.empty_cache()  # ✅ Clears unused memory

import gc
gc.collect()  # ✅ Cleans up Python garbage collector


20

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from tqdm import tqdm
import time
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import json
import torch.backends.cudnn as cudnn
import numpy as np

# Import the I2SB model definition
from i2sb_model import I2SBBackbone  # Ensure you have this import from your previous model file

class TestDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.image_paths = [f for f in os.listdir(data_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform or transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.image_paths[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.image_paths[idx]

class ModelTester:
    def __init__(self, model_path, device, test_loader, output_dir):
        self.device = device
        self.test_loader = test_loader
        self.output_dir = output_dir
        self.model = self.load_model(model_path)
        os.makedirs(output_dir, exist_ok=True)

    def load_model(self, model_path):
        try:
            # Create model first
            model = I2SBBackbone().to(self.device)
            
            # Load state dict 
            checkpoint = torch.load(model_path, map_location=self.device)
            
            # Check if the checkpoint contains the state dict directly or nested
            if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                state_dict = checkpoint['model_state_dict']
            else:
                state_dict = checkpoint
                
            model.load_state_dict(state_dict)
            model.eval()
            print(f"Model loaded successfully from {model_path}")
            return model
        except Exception as e:
            raise Exception(f"Error loading model from {model_path}: {str(e)}")

    def run_test(self):
        results = {
            'psnr_values': [],
            'ssim_values': [],
            'processing_times': [],
            'processed_files': []
        }
    
        # Ensure directories exist to save images
        generated_dir = os.path.join(self.output_dir, 'generated_images_2')
        os.makedirs(generated_dir, exist_ok=True)
    
        total_start_time = time.time()
    
        with torch.no_grad():
            # Use half precision to reduce memory usage
            with torch.cuda.amp.autocast():
                for batch, filenames in tqdm(self.test_loader, desc="Testing"):
                    batch = batch.to(self.device)
    
                    # Process batch and measure time
                    start_time = time.time()
                    outputs = self.model(batch)
                    batch_time = time.time() - start_time
    
                    # Iterate over the batch
                    for i in range(len(batch)):
                        original = batch[i].cpu().numpy().transpose(1, 2, 0)  # Convert to numpy (H, W, C)
                        improved = outputs[i].cpu().numpy().transpose(1, 2, 0)  # Convert to numpy (H, W, C)
    
                        # Denormalize images
                        original = (original * 0.5 + 0.5) * 255
                        improved = (improved * 0.5 + 0.5) * 255
    
                        # Clip and convert to uint8
                        original = np.clip(original, 0, 255).astype(np.uint8)
                        improved = np.clip(improved, 0, 255).astype(np.uint8)
    
                        # Save images
                        original_image_path = os.path.join(generated_dir, f"original_{filenames[i]}")
                        improved_image_path = os.path.join(generated_dir, f"generated_{filenames[i]}")
                        Image.fromarray(original).save(original_image_path)
                        Image.fromarray(improved).save(improved_image_path)
    
                        # Calculate PSNR and SSIM
                        psnr_val = psnr(original, improved, data_range=255.0)
                        ssim_val = ssim(original, improved, channel_axis=-1, data_range=255.0)
    
                        results['psnr_values'].append(psnr_val)
                        results['ssim_values'].append(ssim_val)
                        results['processing_times'].append(batch_time * 1000 / len(batch))  # Convert to ms per image
                        results['processed_files'].append(filenames[i])
    
            total_time = time.time() - total_start_time
    
            # Compile final results
            final_results = {
                'average_psnr': float(np.mean(results['psnr_values'])),
                'average_ssim': float(np.mean(results['ssim_values'])),
                'average_processing_time_ms': float(np.mean(results['processing_times'])),
                'total_processing_time_seconds': total_time,
                'total_patches_processed': len(results['processed_files']),
                'min_psnr': float(np.min(results['psnr_values'])),
                'max_psnr': float(np.max(results['psnr_values'])),
                'min_ssim': float(np.min(results['ssim_values'])),
                'max_ssim': float(np.max(results['ssim_values']))
            }
    
            # Save the results to a JSON file
            with open(os.path.join(self.output_dir, 'test_results.json'), 'w') as f:
                json.dump(final_results, f, indent=4)
    
        return final_results

def main():
    # Configuration - adjust based on your specific paths and requirements
    config = {
        'model_path': r"model-flash.pth",  
        'test_data_path': r"C:\Users\pc\Downloads\patches",  
        'output_dir': 'i2sb_test_results',
        'batch_size': 32  # Adjusted for RTX 3080 Ti
    }

    # Device setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Optimization for GPU
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True

    # Data loading with optimized transform
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    test_dataset = TestDataset(config['test_data_path'], transform=transform)
    test_loader = DataLoader(
        test_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers=4,  # Adjust based on your CPU cores
        pin_memory=True,
        prefetch_factor=2  # Optional: can help with data loading
    )

    # Initialize and run tester
    tester = ModelTester(
        model_path=config['model_path'],
        device=device,
        test_loader=test_loader,
        output_dir=config['output_dir']
    )

    try:
        results = tester.run_test()
        print("\nTest Results:")
        print(f"Average PSNR: {results['average_psnr']:.2f}")
        print(f"Average SSIM: {results['average_ssim']:.2f}")
        print(f"Total processing time: {results['total_processing_time_seconds']:.2f} seconds")
        print(f"Average time per patch: {results['average_processing_time_ms']:.2f} ms")
        print(f"Total patches processed: {results['total_patches_processed']}")
    except Exception as e:
        print(f"Error during testing: {str(e)}")

if __name__ == '__main__':
    main()

C:\Users\pc\anaconda3\lib\site-packages\torch\nn\modules\module.py:1329: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(
C:\Users\pc\AppData\Local\Temp\ipykernel_16312\3853108280.py:83: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Using device: cuda
Model loaded successfully from model-flash.pth


Testing:   0%|                                                                                 | 0/914 [00:00<?, ?it/s]